<a href="https://colab.research.google.com/github/Alex-banzhancha/NumerSense/blob/main/%E2%80%9CNumerSense_T5_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
%%bash
pip install transformers
pip install sentencepiece

In [2]:
import transformers

tokenizer = transformers.T5Tokenizer.from_pretrained('t5-large')
print(tokenizer)

model = transformers.T5ForConditionalGeneration.from_pretrained('t5-large').cuda()


PreTrainedTokenizer(name_or_path='t5-large', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>

In [3]:
input_ids = tokenizer('you may take the subway back and forth to work <extra_id_0> days a week.', return_tensors='pt').input_ids.cuda()

for cand in ['no', 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']:
  labels = tokenizer('<extra_id_0> %s <extra_id_1>' % cand, return_tensors='pt').input_ids.cuda()
  loss = model(input_ids=input_ids, labels=labels).loss
  print(cand, loss)

no tensor(5.5613, device='cuda:0', grad_fn=<NllLossBackward>)
zero tensor(5.2877, device='cuda:0', grad_fn=<NllLossBackward>)
one tensor(4.8512, device='cuda:0', grad_fn=<NllLossBackward>)
two tensor(3.6407, device='cuda:0', grad_fn=<NllLossBackward>)
three tensor(3.3753, device='cuda:0', grad_fn=<NllLossBackward>)
four tensor(3.2980, device='cuda:0', grad_fn=<NllLossBackward>)
five tensor(2.9747, device='cuda:0', grad_fn=<NllLossBackward>)
six tensor(3.1580, device='cuda:0', grad_fn=<NllLossBackward>)
seven tensor(3.1828, device='cuda:0', grad_fn=<NllLossBackward>)
eight tensor(3.7277, device='cuda:0', grad_fn=<NllLossBackward>)
nine tensor(4.0558, device='cuda:0', grad_fn=<NllLossBackward>)
ten tensor(2.9321, device='cuda:0', grad_fn=<NllLossBackward>)


In [5]:
import csv

with open('') as f:
  reader = csv.reader(f, delimiter='\t')
  hit, all = 0, 0
  for row in reader:
    input = row[0].replace('<mask>', '<extra_id_0>')
    input_ids = tokenizer(input, return_tensors='pt').input_ids.cuda()
    losses = {}
    for cand in ['no', 'zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']:
      labels = tokenizer('<extra_id_0> %s <extra_id_1>' % cand, return_tensors='pt').input_ids.cuda()
      loss = model(input_ids=input_ids, labels=labels).loss.item()
      labels2 = tokenizer('<extra_id_0> %s <extra_id_1>' % cand.capitalize(), return_tensors='pt').input_ids.cuda()
      loss2 = model(input_ids=input_ids, labels=labels2).loss.item()
      losses[cand] = (loss + loss2) / 2
    pred = min(losses, key=losses.get)
    ok = int(pred == row[1])
    all += 1
    hit += 1 if ok else 0
    print(ok, pred, input)
  print(hit, all, hit / all)

FileNotFoundError: ignored